In [1]:
from pydantic import BaseModel

In [2]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int
    

In [3]:
p = Person(first_name="John", last_name="John", age=42)

In [4]:
p, p.__dict__

(Person(first_name='John', last_name='John', age=42),
 {'first_name': 'John', 'last_name': 'John', 'age': 42})

In [5]:
from pydantic import ValidationError

try:
    Person(first_name=1234, last_name="12345", age="abcd")
except ValidationError as e:
    print(type(e), e)

<class 'pydantic_core._pydantic_core.ValidationError'> 2 validation errors for Person
first_name
  Input should be a valid string [type=string_type, input_value=1234, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abcd', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing


In [6]:
type(p.first_name), p.first_name

(str, 'John')

In [7]:
p.first_name = "Bob"

In [8]:
p.first_name = 100

In [9]:
p.first_name  # watch out!

100

In [10]:
try:
    Person(first_name=1234, last_name="12345", age="abcd")
except ValidationError as e:
    exceptions = e

In [11]:
exceptions.errors()

[{'type': 'string_type',
  'loc': ('first_name',),
  'msg': 'Input should be a valid string',
  'input': 1234,
  'url': 'https://errors.pydantic.dev/2.8/v/string_type'},
 {'type': 'int_parsing',
  'loc': ('age',),
  'msg': 'Input should be a valid integer, unable to parse string as an integer',
  'input': 'abcd',
  'url': 'https://errors.pydantic.dev/2.8/v/int_parsing'}]

In [12]:
exceptions.json()

'[{"type":"string_type","loc":["first_name"],"msg":"Input should be a valid string","input":1234,"url":"https://errors.pydantic.dev/2.8/v/string_type"},{"type":"int_parsing","loc":["age"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"abcd","url":"https://errors.pydantic.dev/2.8/v/int_parsing"}]'

In [13]:
p = Person.model_validate(dict(first_name="Bob", last_name="Moore", age=33))

In [14]:
p

Person(first_name='Bob', last_name='Moore', age=33)

In [15]:
data_json = '''
{
    "first_name": "Ada",
    "last_name": "Wong",
    "age": 34
}    
'''

p = Person.model_validate_json(data_json)

In [16]:
p

Person(first_name='Ada', last_name='Wong', age=34)

In [17]:
try:
    Person(age=42)
except ValidationError as ex:
    print(ex)

2 validation errors for Person
first_name
  Field required [type=missing, input_value={'age': 42}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
last_name
  Field required [type=missing, input_value={'age': 42}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing


In [18]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int = 0   # default, non required value
    

In [19]:
Person.model_fields

{'first_name': FieldInfo(annotation=str, required=True),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=0)}

In [20]:
p = Person(first_name="John", last_name="Smith")

In [21]:
p

Person(first_name='John', last_name='Smith', age=0)

In [22]:
class Person(BaseModel):
    first_name: str | None = None
    last_name: str
    age: int

In [23]:
Person.model_fields

{'first_name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=True)}

In [24]:
from typing import Union


class Person(BaseModel):
    first_name: Union[str, None] = None
    last_name: str
    age: int = 10


In [25]:
Person.model_fields

{'first_name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=10)}

In [26]:

class Person(BaseModel):
    first_name: str
    last_name: str
    age: int
    lucky_numbers: list[int] | None = None


In [27]:
Person.model_fields

{'first_name': FieldInfo(annotation=str, required=True),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=True),
 'lucky_numbers': FieldInfo(annotation=Union[list[int], NoneType], required=False, default=None)}

In [28]:
p = Person(last_name="Smith", first_name="John", age=22, lucky_numbers=[1, "2", 3.0])

In [29]:
p

Person(first_name='John', last_name='Smith', age=22, lucky_numbers=[1, 2, 3])

In [30]:
from pydantic import Field


class Person(BaseModel):
    id_: int = Field(alias="id")
    first_name: str = Field(alias="First Name")
    last_name: str = Field(alias="LASTNAME")
    age: int = Field(alias="age in years")

In [31]:
p = Person.model_validate({"id": 100, "First Name": "Bob", "LASTNAME": "Mythnic", "age in years": 55})

In [32]:
p

Person(id_=100, first_name='Bob', last_name='Mythnic', age=55)

In [33]:
p.model_dump(), p.model_dump_json()

({'id_': 100, 'first_name': 'Bob', 'last_name': 'Mythnic', 'age': 55},
 '{"id_":100,"first_name":"Bob","last_name":"Mythnic","age":55}')

In [34]:
p.model_dump(by_alias=True)

{'id': 100, 'First Name': 'Bob', 'LASTNAME': 'Mythnic', 'age in years': 55}

In [35]:
p.model_dump_json(by_alias=True)

'{"id":100,"First Name":"Bob","LASTNAME":"Mythnic","age in years":55}'

In [36]:
class Person(BaseModel):
    first_name: str | None = Field(alias="firstName", default=None)
    last_name: str = Field(alias="lastName")

In [37]:
data = {
    "lastName": "Smith"
}

p = Person.model_validate(data)

In [38]:
p

Person(first_name=None, last_name='Smith')

In [39]:
try:
    Person(last_name="Smith")
except ValidationError as e:
    print(e)

1 validation error for Person
lastName
  Field required [type=missing, input_value={'last_name': 'Smith'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing


In [40]:
from pydantic import ConfigDict

In [41]:
class Person(BaseModel):
    model_config = ConfigDict(
        populate_by_name=True,  # allows usage of fields and / or aliases
    )
    
    first_name: str | None = Field(alias="firstName", default=None)
    last_name: str = Field(alias="lastName")
    

In [42]:
p = Person(first_name="John", lastName="Smith")

In [43]:
p

Person(first_name='John', last_name='Smith')

In [44]:
class Model(BaseModel):
    numbers: list[int] = []  # Pydantic allows this - deep copy will be created

In [45]:
m1 = Model()
m2 = Model()

In [46]:
m1.numbers += [1, 2, 3]
m1

Model(numbers=[1, 2, 3])

In [47]:
m2

Model(numbers=[])

In [48]:
from datetime import datetime, timezone

In [49]:
class Log(BaseModel):
    dt: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    message: str

In [50]:
log_1 = Log(message="message 1")
log_2 = Log(message="message 2")

In [51]:
log_1, log_2

(Log(dt=datetime.datetime(2024, 8, 5, 17, 21, 54, 733833, tzinfo=datetime.timezone.utc), message='message 1'),
 Log(dt=datetime.datetime(2024, 8, 5, 17, 21, 54, 733980, tzinfo=datetime.timezone.utc), message='message 2'))

In [52]:
class Model(BaseModel):
    number: float

In [53]:
m = Model(number=1.0)
m.model_dump()

{'number': 1.0}

In [54]:
m = Model(number=1/3)

In [55]:
m

Model(number=0.3333333333333333)

In [56]:
dt = datetime.now(timezone.utc)
dt.isoformat()

'2024-08-05T17:21:54.835924+00:00'

In [57]:
class Model(BaseModel):
    dt: datetime

In [58]:
m = Model(dt=datetime.now(timezone.utc))

In [59]:
m

Model(dt=datetime.datetime(2024, 8, 5, 17, 21, 54, 870357, tzinfo=datetime.timezone.utc))

In [60]:
m.model_dump()

{'dt': datetime.datetime(2024, 8, 5, 17, 21, 54, 870357, tzinfo=datetime.timezone.utc)}

In [61]:
m.model_dump_json()

'{"dt":"2024-08-05T17:21:54.870357Z"}'

In [62]:
from pydantic import field_serializer

In [63]:
class Model(BaseModel):
    number: float

    @field_serializer("number")
    def serialize_float(self, value):
        return round(value, 2)


In [64]:
m = Model(number=1/3)

In [65]:
m.model_dump(), m.model_dump_json()

({'number': 0.33}, '{"number":0.33}')

In [66]:
class Model(BaseModel):
    number: float
    dt: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))

    @field_serializer("number")
    def serialize_float(self, value):
        return round(value, 2)

    @field_serializer("dt", when_used="json-unless-none")  # can also be just "json"
    def serialize_datetime_to_json(self, value):
        return value.strftime("%Y/%-m%-d %I:%M %p")


In [67]:
m = Model(number=1/3)
m.model_dump()

{'number': 0.33,
 'dt': datetime.datetime(2024, 8, 5, 17, 21, 55, 62247, tzinfo=datetime.timezone.utc)}

In [68]:
m.model_dump_json()

'{"number":0.33,"dt":"2024/85 05:21 PM"}'

In [69]:
from pydantic import field_validator

In [70]:
class Model(BaseModel):
    absolute: int

    @field_validator("absolute")  # transform validator - transforms the value, called before instance creation - class method
    @classmethod
    def make_absolute(cls, value):
        # this is an "after validator" - first type validation will be performed
        return abs(value)


In [71]:
m = Model(absolute=-10)
m


Model(absolute=10)

In [72]:
try:
    Model(absolute="abcd")
except ValidationError as e:
    print(e)

1 validation error for Model
absolute
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abcd', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing


In [73]:
class Model(BaseModel):
    numbers: list[int] = []

    @field_validator("numbers")
    @classmethod
    def ensure_unique(cls, numbers):
        if len(set(numbers)) != len(numbers):
            print("Raising the exception")
            raise ValueError("Elements must be unique")
        return numbers


In [74]:
Model(numbers=[1,2,3,4])

Model(numbers=[1, 2, 3, 4])

In [75]:
try:
    Model(numbers=[1,1,1,2,3,4])
except ValidationError as e:
    print(e)

Raising the exception
1 validation error for Model
numbers
  Value error, Elements must be unique [type=value_error, input_value=[1, 1, 1, 2, 3, 4], input_type=list]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error


In [76]:
data = {
    "firstName": "Arthur",
    "lastName": "Clarke",
    "born": {
        "place": {
            "country": "Lunar Colony",
            "city": "Central City"
        },
        "date": "3001-01-01"
    }
}

In [77]:
from datetime import date


class Place(BaseModel):
    country: str
    city: str


class Born(BaseModel):
    place: Place  # model composition, defines nested Pydantic models
    dt: date = Field(alias="date")


class Person(BaseModel):
    first_name: str | None = Field(alias="firstName", default=None)
    last_name: str = Field(alias="lastName")
    born: Born | None = None

In [78]:
arthur = Person.model_validate(data)

In [79]:
arthur.__dict__

{'first_name': 'Arthur',
 'last_name': 'Clarke',
 'born': Born(place=Place(country='Lunar Colony', city='Central City'), dt=datetime.date(3001, 1, 1))}

In [80]:
arthur.born.place.city

'Central City'

In [81]:
arthur.model_dump(), arthur.model_dump_json()

({'first_name': 'Arthur',
  'last_name': 'Clarke',
  'born': {'place': {'country': 'Lunar Colony', 'city': 'Central City'},
   'dt': datetime.date(3001, 1, 1)}},
 '{"first_name":"Arthur","last_name":"Clarke","born":{"place":{"country":"Lunar Colony","city":"Central City"},"dt":"3001-01-01"}}')

In [82]:
print(arthur.model_dump_json(indent=2))

{
  "first_name": "Arthur",
  "last_name": "Clarke",
  "born": {
    "place": {
      "country": "Lunar Colony",
      "city": "Central City"
    },
    "dt": "3001-01-01"
  }
}
